#1] Imports

In [1]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 2] Pre-Processing

In [2]:
# Reading in relevant data
data = pd.read_csv("../Data/WildfireData.csv", na_values="NaN")
data = data[['fire_size', 'latitude', 'longitude', 'discovery_month', 'Vegetation', 'Temp_pre_7', 'Hum_pre_7', 'Prec_pre_7', 'Wind_pre_7']]

In [3]:
# Drop NA
data = data.dropna()

In [4]:
# One Hot Encodings
non_dummy_cols = ['fire_size', 'latitude','longitude', 'Temp_pre_7', 'Hum_pre_7', 'Prec_pre_7', 'Wind_pre_7'] 
dummy_cols = list(set(data.columns) - set(non_dummy_cols))
data = pd.get_dummies(data, columns=dummy_cols)

# 3] Create Individual Models 

In [5]:
#Split train/test
from sklearn.model_selection import train_test_split
X = data.drop('fire_size', axis=1) 
y = data.fire_size  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [6]:
#Imports for various models
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from xgboost import XGBRegressor

seed = 43

In [7]:
#Initialize Models
elnt = ElasticNet(random_state = seed)
svm = SVR()
xgb = XGBRegressor(random_state = seed, n_jobs = -1)

elnt.fit(X_train, y_train)
svm.fit(X_train, y_train)
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=43,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
#Hyperparameter grid search
from sklearn.model_selection import GridSearchCV
def grid_search_cv(model, params):
    global best_params, best_score
    grid_search = GridSearchCV(estimator = model, param_grid = params, cv = 10, verbose = 1,
                            scoring = 'neg_mean_squared_error', n_jobs = -1)
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_ 
    best_score = np.sqrt(-1*(np.round(grid_search.best_score_, 5)))
    return best_params, best_score

In [ ]:
#Optimize Elastic Net
elastic_params = {'alpha': [0.0005, 0.001, 0.002, 0.005], 
                 'l1_ratio': [0.60, 0.65, 0.70, 0.80],
                 'random_state':[seed]}
grid_search_cv(elnt, elastic_params)
elastic_best_params, elastic_best_score = best_params, best_score
print('Elastic Net best params:{} & best_score:{:0.5f}' .format(elastic_best_params, elastic_best_score))

Fitting 10 folds for each of 16 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   33.8s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  1.7min finished


Elastic Net best params:{'alpha': 0.005, 'l1_ratio': 0.6, 'random_state': 43} & best_score:13879.59241


In [ ]:
#Optimize Support Vector Regressor
svm_params = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 
    'C': [4, 5], 
    'gamma':[0.0001, 0.001]}

grid_search_cv(svm, svm_params)
svm_best_params, svm_best_score = best_params, best_score
print('SVM best params:{} & best_score:{:0.5f}' .format(svm_best_params, svm_best_score))

In [8]:
#Optimize XGB
xgb_opt = XGBRegressor(colsample_bytree = 0.4603, gamma = 0.0468, 
                             learning_rate = 0.04, max_depth = 3, 
                             min_child_weight = 1.7817, n_estimators = 2500,
                             reg_alpha = 0.4640, reg_lambda = 0.8571,
                             subsample = 0.5213, silent = 1,
                             nthread = -1, random_state = 7)

In [9]:
#Retrain with optimized hyperparameters
# elastic_net_opt = ElasticNet(**elastic_best_params)
# svm_opt = SVR(**svm_best_params)
xgb_opt = xgb_opt

# elastic_net_opt.fit(X_train, y_train)
# svm_opt.fit(X_train, y_train)
xgb_opt.fit(X_train, y_train)

[14:55:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.4603, gamma=0.0468,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.04, max_delta_step=0, max_depth=3,
             min_child_weight=1.7817, missing=nan, monotone_constraints='()',
             n_estimators=2500, n_jobs=4, nthread=-1, num_parallel_tree=1,
             random_state=7, reg_alpha=0.464, reg_lambda=0.8571,
             scale_pos_weight=1, silent=1, subsample=0.5213,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
#Define learning curve function
from sklearn.model_selection import learning_curve
def plot_learning_curve(model):
    train_sizes, train_scores, test_scores = learning_curve(model, X_train, y_train, 
                                            train_sizes = np.linspace(0.01, 1.0, 20), cv = 10, scoring = 'neg_mean_squared_error', 
                                            n_jobs = -1, random_state = seed)
    train_mean = np.mean(train_scores, axis = 1)
    train_std = np.std(train_scores, axis = 1)

    test_mean = np.mean(test_scores, axis = 1)
    test_std = np.std(test_scores, axis = 1)  
    
    plt.plot(train_sizes, train_mean, 'o-', color = 'red',  label = 'Training score')
    plt.plot(train_sizes, test_mean, 'o-', color = 'green', label = 'Cross-validation score')
    
    plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha = 0.1, color = 'r') # Alpha controls band transparency.
    plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha = 0.1, color = 'g')
    
    font_size = 12
    plt.xlabel('Training Set Size', fontsize = font_size)
    plt.ylabel('Accuracy Score', fontsize = font_size)
    plt.xticks(fontsize = font_size)
    plt.yticks(fontsize = font_size)
    plt.legend(loc = 'best')
    plt.grid()


#4] Combine into Ensemble Model

In [ ]:
#Dataframe containing all 3 ensemble models
base_model_prediction = pd.DataFrame({
       'Elastic Net':elnt.predict(X_test), 
       'SVM':svm.predict(X_test),
       'XGB':xgb.predict(X_test), 
  })

from IPython.display import Markdown
def bold(string):
    display(Markdown(string))
bold('**All the Base Model Predictions:**')
display(base_model_prediction.head())

**All the Base Model Predictions:**

,Elastic Net,SVM,XGB
0,5376.199466,10.471358,6053.119141
1,1548.151290,3.253075,145.265396
2,1125.510421,3.248031,232.562973
3,6157.072399,9.606693,-13.402963
4,4800.066478,13.911445,5452.743164


In [ ]:
#Averaging Models
y_elastic = elnt.predict(X_test)
y_svm = svm.predict(X_test)
y_xgb = xgb.predict(X_test)

avg_ensemble = (y_svm + y_xgb + y_elastic)/3
final = pd.DataFrame({'Actual': y_test, 'Predicted': avg_ensemble})
print(final)

        Actual     Predicted
54607  61416.0   3256.808049
39066      2.5    155.583506
52438  29229.5  15241.124802
269      168.3   2641.811681
37095      2.5    833.535488
...        ...           ...
15793   1201.8    624.305088
39713      2.0   -571.971091
42309      2.0   1936.209311
1990       1.0   -722.056072
3059       2.0    846.890622

[16611 rows x 2 columns]


In [1]:
# from joblib import dump, load
# dump(elnt, 'elnt.joblib')
# dump(svm, 'svm.joblib')
# dump(xgb, 'xgb.joblib')
xgb_opt.save_model('../Weights/xgb.json')

NameError: name 'xgb' is not defined